In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [31]:
json_file_path = os.path.join("./annotations", "bapproach1.json")
with open(json_file_path, 'r') as f:
    single_json_data = json.load(f)

single_json_data
categories = single_json_data["categories"]
annotations = single_json_data["annotations"]
categories
annotations
categories[annotations[0]["category_id"]-1]["name"]

'approach'

In [43]:
import os
import json
import pandas as pd

json_folder = "./annotations"
csv_folder = "./output"
output_folder = "./labeled_output"

os.makedirs(output_folder, exist_ok=True)

for json_file in os.listdir(json_folder):
    if json_file.endswith(".json"):
        json_path = os.path.join(json_folder, json_file)
        csv_path = os.path.join(csv_folder, json_file.replace(".json", ".csv"))
        
        if not os.path.exists(csv_path):
            print(f"El archivo CSV para {json_file} no existe. Saltando...")
            continue
        
        with open(json_path, 'r') as f:
            json_data = json.load(f)

        categories = json_data["categories"]
        
        annotated_frames = {ann["image_id"] for ann in json_data["annotations"]}
        annotated_frames_categories = {ann["category_id"] for ann in json_data["annotations"]}

        csv_data = pd.read_csv(csv_path)

        csv_data["annotation"] = csv_data["frame"].apply(
            lambda frame: "Still" if frame not in annotated_frames else categories[json_data["annotations"][list(annotated_frames)[0]]["category_id"]-1]["name"]
        )
        
        output_csv_path = os.path.join(output_folder, os.path.basename(csv_path))
        csv_data.to_csv(output_csv_path, index=False)
        print(f"Procesado: {json_file} -> {output_csv_path}")

print("Procesamiento completado.")

Procesado: bapproach1.json -> ./labeled_output\bapproach1.csv
Procesado: bapproach2.json -> ./labeled_output\bapproach2.csv
Procesado: bapproach3.json -> ./labeled_output\bapproach3.csv
Procesado: bback1.json -> ./labeled_output\bback1.csv
Procesado: bback2.json -> ./labeled_output\bback2.csv
Procesado: bback3.json -> ./labeled_output\bback3.csv
Procesado: bjump1.json -> ./labeled_output\bjump1.csv
Procesado: bjump2.json -> ./labeled_output\bjump2.csv
Procesado: bjump3.json -> ./labeled_output\bjump3.csv
Procesado: blturn1.json -> ./labeled_output\blturn1.csv
Procesado: blturn2.json -> ./labeled_output\blturn2.csv
Procesado: blturn3.json -> ./labeled_output\blturn3.csv
Procesado: blwalk1.json -> ./labeled_output\blwalk1.csv
Procesado: blwalk2.json -> ./labeled_output\blwalk2.csv
Procesado: brturn1.json -> ./labeled_output\brturn1.csv
Procesado: brturn2.json -> ./labeled_output\brturn2.csv
Procesado: brturn3.json -> ./labeled_output\brturn3.csv
Procesado: brwalk1.json -> ./labeled_outpu